<a href="https://colab.research.google.com/github/dnmuturi/SIT742/blob/main/SIT742Task2code_test_NM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**END TERM ASSIGNMENT**

 **GROUP C30 Members**

1. David Muturi - S225177509
2. Nhlanhla Matukane - S225177376
3. Vincent Nwobi - S225177483

### **Part 1**

**Answer 1.1**

In [1]:
#install the spark library
!pip install pyspark
!apt-get install pandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
pandoc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
#load necessary libraries
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
import pandas as pd
from pyspark.sql.functions import col, from_unixtime, to_date, when, lit, trim, count, date_format,avg,struct, asc
from pyspark.sql.functions import lower, regexp_replace, split, explode, year, concat_ws, length, collect_list,rank,countDistinct, month, dayofweek
from pyspark.sql.window import Window
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import hash
import re
from collections import Counter
from wordcloud import WordCloud
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
from itertools import product

In [3]:
# Mount Google Drive
drive.mount('/content/drive')
# Define the path to the CSV file
csv_path = '/content/drive/My Drive/SIT742/review.csv'
# Define the path to the second CSV file
meta_csv_path = '/content/drive/My Drive/SIT742/meta-review-business.csv'

ValueError: mount failed

In [ ]:
from google.colab import files
import pandas as pd

# Once uploaded, the files exist on the server, and you can now use their names directly
csv_path = 'review.csv'
meta_csv_path = 'meta-review-business.csv'

# Now you can load them
df_reviews = pd.read_csv(csv_path)
df_meta = pd.read_csv(meta_csv_path)

print("\nFiles successfully uploaded and loaded!")
df_reviews.head()
df_meta.head()

In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("ReviewAnalysis").getOrCreate()

# --- Load review.csv ---
print("Loading 'review.csv' into a Spark DataFrame...")
try:
    # Use the path from your file upload cell: 'review.csv'
    df_review = spark.read.csv(
        csv_path,
        header=True,
        inferSchema=True,
        sep=',',
        quote='"',
        escape='"',
        multiLine=True
    )
    print("review.csv' loaded successfully!")
    df_review.show(5, truncate=False)  # Display the first five rows without truncating

except Exception as e:
    print(f"An error occurred while loading 'review.csv': {e}")


# --- Load meta-review-business.csv ---
print("\nLoading 'meta-review-business.csv' into a Spark DataFrame...")
try:
    # Use the path from your file upload cell: 'meta-review-business.csv'
    df_meta= spark.read.csv(
        meta_csv_path,
        header=True,
        inferSchema=True,
        sep=',',
        quote='"',
        escape='"',
        multiLine=True
    )
    print("meta-review-business.csv' loaded successfully!")
    df_meta.show(5, truncate=False)

except Exception as e:
    print(f"An error occurred while loading 'meta-review-business.csv': {e}")

In [ ]:
# evaluation for df_review
print("--- DataFrame: df_review ---")
total_reviews = df_review.count()
print(f"Total rows: {total_reviews:,}")
print("\nSchema:")
df_review.printSchema()

In [ ]:
#show summary statistics
df_review.describe().show()

In [ ]:
# Evaluation for df_meta
print("\n\n DataFrame: df_meta")
total_businesses = df_meta.count()
print(f"Total rows: {total_businesses:,}")
print("\nSchema:")
df_meta.printSchema()

In [ ]:
# evaluation for df_review
print("--- DataFrame: df_review ---")
total_reviews = df_review.count()
print(f"Total rows: {total_reviews:,}")
print("\nSchema:")
df_review.printSchema()

In [ ]:
# --- Statistics for df_meta ---
print("\n Summary Statistics for df_meta")
df_meta.describe().show()

In [ ]:
# Before any further analysis, we evaluated the missing values in the dataset and used the Spark DataFrame functions to count them. The normal way to deal with missing values is to treat nulls as missing, empty strings as missing for text, and NaN as missing for numbers (Little and Rubin, 2019; van Buuren, 2018).
# We do this with Apache Spark SQL functions and data types. The meanings of isNull, isnan, and trim are in the Spark docs.These are consistent with SQL-style null handling and are in line with how SQL handles nulls (Apache Spark, 2025).

from pyspark.sql.functions import col, trim, when, isnan, lit, sum as F_sum, count as F_count
from pyspark.sql.types import StringType, NumericType

import pandas as pd

def missing_profile_spark(df, df_name="dataframe"):
    """
    Build a tidy table with missingness and completeness per column for a Spark DataFrame.
    Returns a pandas.DataFrame for easy viewing and saving.
    """
    # Basic checks
    total_rows = df.count()
    if total_rows == 0:
        print(f"[{df_name}] has 0 rows. Nothing to profile.")
        return pd.DataFrame(columns=["column", "dtype", "missing_count", "missing_rate",
                                     "non_missing_count", "completeness_rate"])

    # Build "missing flag" per column in one pass to be efficient
    cols = []
    dtypes = []
    missing_flags = []

    for f in df.schema.fields:
        c = f.name
        dt = f.dataType
        cols.append(c)
        dtypes.append(str(dt))

        # Define what counts as missing for this column
        is_null = col(c).isNull()
        if isinstance(dt, StringType):
            is_blank = (trim(col(c)) == "")
            flag = when(is_null | is_blank, 1).otherwise(0)
        elif isinstance(dt, NumericType):
            flag = when(is_null | isnan(col(c)), 1).otherwise(0)
        else:
            # For non-string, non-numeric types, we count only null as missing
            flag = when(is_null, 1).otherwise(0)

        missing_flags.append(F_sum(flag).alias(c))

    # Aggregate all missing counts in one go
    counts_row = df.agg(*missing_flags).collect()[0].asDict()

    # Build a tidy pandas table
    rows = []
    for c, dt in zip(cols, dtypes):
        miss = int(counts_row.get(c, 0))
        miss_rate = miss / float(total_rows)
        non_miss = total_rows - miss
        comp_rate = 1.0 - miss_rate
        rows.append({
            "column": c,
            "dtype": dt,
            "missing_count": miss,
            "missing_rate": round(miss_rate, 6),
            "non_missing_count": non_miss,
            "completeness_rate": round(comp_rate, 6)
        })

    out = pd.DataFrame(rows).sort_values(["missing_rate", "missing_count", "column"], ascending=[False, False, True]).reset_index(drop=True)

    # Print a compact summary and return the full table
    print(f"\n[{df_name}] rows: {total_rows}, columns: {len(cols)}")
    print("Top columns by missing_rate:")
    display(out.head(10))
    return out

# Run profiles for both data sets
review_profile = missing_profile_spark(df_review, df_name="df_review")
meta_profile   = missing_profile_spark(df_meta,   df_name="df_meta")


# References (Harvard style):
# Apache Spark (2025) Spark SQL, Built-in Functions. Available at: https://spark.apache.org/docs/latest/api/python/ (Accessed: 03/10/2025).
# Little, R.J.A. and Rubin, D.B. (2019) Statistical Analysis with Missing Data. 3rd edn. Hoboken, NJ: Wiley.
# van Buuren, S. (2018) Flexible Imputation of Missing Data. 2nd edn. Boca Raton, FL: CRC Press. Available at: https://stefvanbuuren.name/fimd/ (Accessed: 03/10/2025).

In [ ]:
# df_review — missingness bar plot
# Idea:
# - Convert a small slice from Spark to pandas (for plotting).
# - Treat empty strings "" as missing by converting them to NaN.
# - Plot a single missingness bar chart to see gaps per column.

!pip -q install missingno

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno

# Convert up to N rows to pandas
N =  521515  # number of observations
pdf_review = df_review.limit(N).toPandas()

# Empty strings -> NaN so they count as missing for text columns
pdf_review = pdf_review.replace(r"^\s*$", np.nan, regex=True)

# One clear plot
plt.figure()
msno.bar(pdf_review)
plt.title("df_review - missingness by column (bar)")
plt.show()

# Quick text summary (top 10 by missing rate)
print("\nTop 10 columns by missing rate (sampled):")
print(pdf_review.isna().mean().sort_values(ascending=False).head(10).round(3))


In [ ]:
# df_meta — missingness bar plot
# Same approach as df_review to keep things consistent.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno

# Convert up to N rows to pandas
N = 12774 # number of observations
pdf_meta = df_meta.limit(N).toPandas()

# Empty strings -> NaN
pdf_meta = pdf_meta.replace(r"^\s*$", np.nan, regex=True)

# One clear plot
plt.figure()
msno.bar(pdf_meta)
plt.title("df_meta - missingness by column (bar)")
plt.show()

# Quick text summary (top 10 by missing rate)
print("\nTop 10 columns by missing rate (sampled):")
print(pdf_meta.isna().mean().sort_values(ascending=False).head(10).round(3))

# References (Harvard style):
# Bilogur, A. (2018) missingno: a missing data visualization suite. Available at: https://github.com/ResidentMario/missingno (Accessed: 04/10/2025).
# Little, R.J.A. and Rubin, D.B. (2019) Statistical Analysis with Missing Data. 3rd edn. Hoboken, NJ: Wiley.
# van Buuren, S. (2018) Flexible Imputation of Missing Data. 2nd edn. Boca Raton, FL: CRC Press. Available at: https://stefvanbuuren.name/fimd/ (Accessed: 04/10/2025).


**Answer 1.1.1**

In [ ]:
#function to count rows that are none or null
def count_empty_rows(df):
  # Filter rows where 'text' column is null or an empty string
  null_or_empty_text_count = df.filter(
    col("text").isNull() | (trim(col("text")) == "")
  ).count()
  return null_or_empty_text_count
#show count of empty rows in text field before filling in with no review
print(f"The number of empty rows in the text column before replacing with 'no review' is: {count_empty_rows(df_review)}")

In [ ]:
# Replace null values with "No review"
df_review = df_review.fillna({'text': 'no review'})

# Replace empty strings (after trimming whitespace) with "No review"
df_review = df_review.withColumn("text",
    when(trim(col("text")) == "", lit("no review")).otherwise(col("text"))
)

# Show the count of text column which is empty or null to verify the changes
print(f"The number of empty rows in the text column after replacing with no review is: {count_empty_rows(df_review)}")

# Count rows where the 'text' column is 'no review'
no_review_count = df_review.filter(col("text") == "no review").count()

print(f"The number of reviews with 'no review' in the text column is: {no_review_count}")


**Answer 1.1.2**

In [ ]:
# Set the legacy time parser policy to corrected to handle potential parsing issues
spark.conf.set("spark.sql.legacy.timeParserPolicy", "CORRECTED")

# Convert 'time' from epoch milliseconds to a timestamp, then to a date string
df_review = df_review.withColumn(
    "newtime",
    to_date(from_unixtime(col("time") / 1000).cast("timestamp"), "yyyy-MM-dd")
)

# Display the first few rows with the new column
df_review.select("user_id","name","rating","time", "newtime","text","gmap_id").show(5)

**Answer 1.2**

**Answer 1.2.1**

In [ ]:
# Calculate the number of reviews per unique gmap_id
reviews_per_gmap = df_review.groupBy("gmap_id").agg(count("*").alias("review_count"))

# Cast the review_count to float type
reviews_per_gmap = reviews_per_gmap.withColumn("review_count", col("review_count").cast(DoubleType()))

# Show the top 5 results
print("Number of reviews per unique gmap_id (Top 5):")
reviews_per_gmap.orderBy(col("review_count").desc()).show(5)

**Answer 1.2.2**

In [ ]:
# Convert PySpark DataFrame to pandas DataFrame
df = df_review.toPandas()

# Create 'review_time' column at the hour level
df['review_time'] = pd.to_datetime(df['time'], unit='ms').dt.hour

# Display the first 5 rows of the pandas DataFrame
print("Pandas DataFrame with 'review_time' column (Top 5):")
display(df.head())

**Answer 1.2.3**

In [ ]:
# Analyze the distribution of reviews by hour
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='review_time', palette='viridis')
plt.title('Distribution of Review Times by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Reviews')
plt.xticks(range(0, 24))
plt.grid(axis='y')
plt.show()

# Analyze the number of businesses reviewed at different times
unique_gmap_ids_per_hour = df.groupby('review_time')['gmap_id'].nunique().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(data=unique_gmap_ids_per_hour, x='review_time', y='gmap_id', palette='viridis')
plt.title('Number of Unique Businesses Reviewed per Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Unique Businesses')
plt.xticks(range(0, 24))
plt.grid(axis='y')
plt.show()

### Insights from Visualizations

The visualizations above provide insights into the review patterns based on the time of day:

**Distribution of Review Times by Hour:**
- The first plot shows the total number of reviews submitted at each hour of the day.
- We can observe that there is a peak in reviews during certain hours, indicating when users are most active in leaving feedback. This could be related to business operating hours, user routines, or other factors.

**Number of Unique Businesses Reviewed per Hour:**
- The second plot shows the number of distinct businesses (identified by `gmap_id`) that received at least one review during each hour.
- This helps understand if reviews are spread across many businesses throughout the day or concentrated on a few during specific times.

Further analysis could involve looking at the average rating per hour, the types of businesses reviewed at different times, or correlating review times with other factors in the dataset.

**Answer 1.3**

**Answer 1.3.1**

In [ ]:
# rename the name column in the meta_data to avoid duplicate name error with the reviews data set
df_meta = df_meta.withColumnRenamed("name", "business_name")

# Join the two DataFrames on the 'gmap_id' column
joined_df = df_review.join(df_meta, on="gmap_id", how="inner")

# Show the schema and some rows of the joined DataFrame
print("Joined DataFrame Schema:")
joined_df.printSchema()

print("Joined DataFrame (Top 5):")
joined_df.show(5, truncate=False)

In [ ]:
# Extract the day of the week from the 'newtime' column
joined_df_with_day = joined_df.withColumn("day_of_week", date_format(col("newtime"), "E"))

# Group by day of the week and count the reviews
reviews_by_day = joined_df_with_day.groupBy("day_of_week").count()

# Define the order of the days of the week for plotting
day_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

# Convert to pandas DataFrame for plotting
reviews_by_day_pd = reviews_by_day.toPandas()

# Sort the pandas DataFrame by the defined day order
reviews_by_day_pd['day_of_week'] = pd.Categorical(reviews_by_day_pd['day_of_week'], categories=day_order, ordered=True)
reviews_by_day_pd = reviews_by_day_pd.sort_values('day_of_week')


# Plot the results as a line chart
plt.figure(figsize=(10, 6))
sns.lineplot(data=reviews_by_day_pd, x='day_of_week', y='count', marker='o')
plt.title('Number of Reviews per Day of the Week (Joined Data)')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Reviews')
plt.grid(axis='y')
plt.show()

#### **Observations**

The day of the week that has the highest reviews is Sunday, Friday has the lowest reviews.

**Answer 1.3.2**

In [ ]:
# Find the day with the maximum number of reviews
peak_day = reviews_by_day_pd.loc[reviews_by_day_pd['count'].idxmax()]['day_of_week']
print(f"The workday with the most reviews is: {peak_day}")

# Filter the joined DataFrame for the peak day
peak_day_df = joined_df_with_day.filter(col("day_of_week") == peak_day)

# Group by business name and category and calculate the average rating
avg_rating_by_business = peak_day_df.groupBy("business_name", "category").agg(avg("rating").alias("average_rating"))

# Find the business(es) with the highest average rating on the peak day
highest_rated_businesses = avg_rating_by_business.orderBy(col("average_rating").desc()).limit(5)

print(f"\nBusinesses with the highest average rating on {peak_day} (Top 5):")
highest_rated_businesses.show(truncate=False)

**Answer 1.3.3**

In [ ]:
# Group by category and count the number of businesses
category_counts = joined_df.groupBy("category").count()

# Order by count in descending order
category_counts = category_counts.orderBy(col("count").desc())

# Convert to pandas DataFrame
category_counts_pd = category_counts.toPandas()

# Select the top 10 categories
top_10_categories = category_counts_pd.head(10)

# Create a bar plot
plt.figure(figsize=(12, 7))
sns.barplot(data=top_10_categories, x='category', y='count', palette='viridis')
plt.title('Top 10 Business Categories by Count')
plt.xlabel('Business Category')
plt.ylabel('Number of Businesses')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Get the list of top 10 categories
top_categories_list = top_10_categories['category'].tolist()

# Filter the joined DataFrame to include only the top 10 categories
filtered_df = joined_df_with_day.filter(col("category").isin(top_categories_list))

# Extract the hour of the day from the 'newtime' column
filtered_df = filtered_df.withColumn("review_hour", date_format(col("newtime"), "H"))

# Show the schema and some rows of the filtered DataFrame with the new column
print("Filtered DataFrame Schema:")
filtered_df.printSchema()

print("Filtered DataFrame (Top 5):")
filtered_df.show(5, truncate=False)

In [ ]:
# Group by category and review_hour and count the number of reviews
reviews_by_category_hour = filtered_df.groupBy("category", "review_hour").count()

# Find the peak review hour for each category

window_spec = Window.partitionBy("category").orderBy(col("count").desc())

ranked_reviews = reviews_by_category_hour.withColumn("rank", rank().over(window_spec))

peak_hours_per_category = ranked_reviews.filter(col("rank") == 1)

# Display the peak review hour for each of the top 10 categories
print("Peak review hour for each of the top 10 categories:")
peak_hours_per_category.orderBy(col("count").desc()).show(truncate=False)

## Analyze average ratings by category


In [ ]:
# Group by category and calculate the average rating
avg_rating_by_category = joined_df.groupBy("category").agg(avg("rating").alias("average_rating"))

# Order by average_rating in descending order
avg_rating_by_category = avg_rating_by_category.orderBy(col("average_rating").desc())

# Convert to pandas DataFrame
avg_rating_by_category_pd = avg_rating_by_category.toPandas()

# Select the top 10 categories by average rating
top_10_avg_rating_categories = avg_rating_by_category_pd.head(10)

# Create a bar plot
plt.figure(figsize=(20, 15))
sns.barplot(data=top_10_avg_rating_categories, x='category', y='average_rating', palette='viridis')
plt.title('Top 10 Business Categories by Average Rating')
plt.xlabel('Business Category')
plt.ylabel('Average Rating')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

**Analysis Summary:**

Business Categories, Peak Hours, and Average Ratings
This analysis explored review patterns based on business categories, focusing on the distribution of businesses, peak review hours for top categories, and average ratings across categories.

**Distribution of Businesses Across Categories:**

Based on the analysis of business categories, the most frequent categories in the dataset are:

['Shopping mall']
['Fast food restaurant', 'Breakfast restaurant', 'Hamburger restaurant', 'Restaurant', 'Sandwich shop']
['Department store', 'Clothing store', 'Craft store', 'Shoe store', 'Sporting goods store']
['Grocery store', 'Grocery delivery service']
['Mexican restaurant']
['Fast food restaurant', 'Breakfast restaurant', 'Burger joint', 'Restaurant', 'Sandwich shop']
['Restaurant']
['Warehouse store', 'Department store']
['American restaurant']
['Gas station']
The bar plot titled "Top 10 Business Categories by Count" visually represents this distribution, clearly showing the categories with the highest number of associated businesses.

**Peak Review Hours for Top Categories:**

The analysis of peak review hours for the top 10 business categories revealed that for all of these categories, the hour with the highest number of reviews is '0' (which likely corresponds to midnight or the beginning of the day, depending on the exact time data representation).

**Average Ratings by Category:**

The calculation of average ratings by category identified the categories with the highest average ratings. The top 10 categories by average rating are:

['Beauty salon', 'Hair care', 'Hair salon', 'Nail salon']
['Snowboard shop', 'Clothing store', 'Shoe store', 'Ski shop', 'Sporting goods store', 'Store']
['Horseback riding service', 'Bed & breakfast', 'Cabin rental agency', 'Cottage', 'Guest house']
['Pediatric dentist', 'Dentist']
['Pediatrician']
['Surgeon', 'Doctor', 'Medical spa']
['Waterfall', 'Tourist attraction']
['Rafting', 'Raft trip outfitter', 'Scenic spot']
['Dogsled ride service', 'Helicopter tour agency', 'Tour operator']
['Leather goods store', 'Store']
The bar plot titled "Top 10 Business Categories by Average Rating" illustrates these top-rated categories and their corresponding average ratings.



In [ ]:
# Combine all review text into a single string in PySpark
all_reviews_text_df = joined_df.select(concat_ws(" ", col("text"))).collect()
all_reviews_text = all_reviews_text_df[0][0] if all_reviews_text_df else ""


# Remove punctuation and convert to lowercase using PySpark functions
words_df = joined_df.select(lower(regexp_replace(col("text"), r'[^\w\s]', '')).alias("text"))
words_df = words_df.select(explode(split(col("text"), "\s+")).alias("word"))

# Remove common English stop words using PySpark
stop_words = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "herself",
    "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what",
    "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was",
    "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did",
    "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until",
    "while", "of", "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
    "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
    "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", "no", "review"
])

words_df = words_df.filter(~col("word").isin(stop_words)).filter(length(col("word")) > 1)


# Count the frequency of each word using PySpark
word_counts_df = words_df.groupBy("word").count().orderBy(col("count").desc())

# Get the top 30 most common words
top_30_words = word_counts_df.limit(30).collect()

print("Top 30 most common words:")
for row in top_30_words:
    print(f"{row['word']}: {row['count']}")

# Generate word clouds for each year
# Extract the year from the 'newtime' column
reviews_with_year = joined_df.withColumn("review_year", year(col("newtime")))

# Group by year and concatenate text for each year
reviews_by_year_df = reviews_with_year.groupBy("review_year").agg(concat_ws(" ", collect_list(col("text"))).alias("all_text"))

# Collect the data for word cloud generation
reviews_by_year_pd = reviews_by_year_df.toPandas()

In [ ]:
#sort reviews by year before plotting the word clouds
reviews_by_year_pd.sort_values(by='review_year', inplace=True)
reviews_by_year_pd.head()

In [ ]:
# Generate word cloud for each year
num_years = len(reviews_by_year_pd)
num_cols = 2
num_rows = (num_years + num_cols - 1) // num_cols # Calculate number of rows needed

fig, axes = plt.subplots(num_rows, num_cols, figsize=(10 * num_cols, 5 * num_rows))
axes = axes.flatten() # Flatten the array of axes for easy iteration

for i, (index, row) in enumerate(reviews_by_year_pd.iterrows()):
    review_year_val = row['review_year']
    text = row['all_text']

    if text: # Check if there is any text for the year
        # Remove punctuation and convert to lowercase
        text = re.sub(r'[^\w\s]', '', text).lower()
        words = text.split()
        words = [word for word in words if word not in stop_words and len(word) > 1]
        text = " ".join(words)

        if text: # Check if there are any words left after removing stop words
            wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
            axes[i].imshow(wordcloud, interpolation='bilinear')
            axes[i].axis('off')
            axes[i].set_title(f'Word Cloud for {review_year_val}')
        else:
            axes[i].axis('off') # Hide axis if no words to display
            axes[i].set_title(f'No significant words for {review_year_val}')
    else:
        axes[i].axis('off') # Hide axis if no text for the year
        axes[i].set_title(f'No reviews for {review_year_val}')

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

### Insights from Top Words by Year

Analyzing the top words for each year provides a clearer view of the review content and highlights shifts in focus over time:

*   **Consistency of Core Themes:** Words like "great," "good," "food," "place," and "service" consistently appear in the top words across many years, indicating that these are fundamental aspects of the reviewed businesses that users frequently comment on.
*   **Growth in Review Volume:** The increasing counts for the top words over the years (especially noticeable from 2016 onwards) directly reflect the significant growth in the number of reviews in the dataset during this period.
*   **Dominance of Positive Language:** The prevalence of positive words like "great," "good," and "amazing" in the top lists suggests a generally positive sentiment within the reviews, based on the most frequent terms.
*   **Specific Mentions:** While the very top words are quite general, looking beyond the top 5 or 10 for each year might reveal more specific keywords related to particular types of businesses or events that were prominent in those years.


**Answer 1.5**

In [ ]:
# Determine the number of unique reviewers per business
unique_reviewers_per_business = joined_df.groupBy("business_name", "gmap_id").agg(countDistinct("user_id").alias("unique_reviewer_count"))

print("Number of unique reviewers per business (Top 10):")
unique_reviewers_per_business.orderBy(col("unique_reviewer_count").desc()).show(10, truncate=False)

# Determine the number of unique reviewers per category
unique_reviewers_per_category = joined_df.groupBy("category").agg(countDistinct("user_id").alias("unique_reviewer_count"))

print("Number of unique reviewers per category (Top 10):")
unique_reviewers_per_category.orderBy(col("unique_reviewer_count").desc()).show(10, truncate=False)

# Analyze temporal patterns of reviews by year
reviews_by_year = joined_df.withColumn("review_year", year(col("newtime"))).groupBy("review_year").count().orderBy("review_year")

print("Number of reviews per year:")
reviews_by_year.show(truncate=False)

# Analyze temporal patterns of reviews by month
reviews_by_month = joined_df.withColumn("review_month", month(col("newtime"))).groupBy("review_month").count().orderBy("review_month")

print("Number of reviews per month:")
reviews_by_month.show(truncate=False)

# Analyze temporal patterns of reviews by day of the week
reviews_by_dayofweek = joined_df.withColumn("review_dayofweek", dayofweek(col("newtime"))).groupBy("review_dayofweek").count().orderBy("review_dayofweek")

print("Number of reviews per day of the week (1 = Sunday, 7 = Saturday):")
reviews_by_dayofweek.show(truncate=False)

### Insights from Unique Reviewers and Temporal Patterns

**Unique Reviewers per Business and Category:**

*   The analysis of unique reviewers per business reveals the businesses that have attracted the largest number of distinct individuals to leave reviews. The top businesses with the highest unique reviewer counts are:
    *   Moose's Tooth Pub & Pizzeria
    *   Dimond Center
    *   Walmart Supercenter (multiple locations appear in the top 10)
    *   Costco Wholesale (multiple locations appear in the top 10)
    *   Anchorage 5th Avenue Mall
    *   49th State Brewing - Anchorage
    *   Tikahtnu Commons

*   Similarly, examining unique reviewers per category highlights the business categories that receive reviews from the broadest range of users. The top categories by unique reviewer count are:
    *   ['Shopping mall']
    *   ['Department store', 'Clothing store', 'Craft store', 'Discount store', 'Electronics store', 'Grocery store', 'Home goods store', 'Sporting goods store', 'Supermarket', 'Toy store']
    *   ['Fast food restaurant', 'Breakfast restaurant', 'Coffee shop', 'Hamburger restaurant', 'Restaurant', 'Sandwich shop']
    *   ['Grocery store', 'Grocery delivery service']
    *   ['Mexican restaurant']
    *   ['Warehouse store', 'Department store']
    *   ['Fast food restaurant', 'Breakfast restaurant', 'Burrito restaurant', 'Lunch restaurant', 'Takeout Restaurant', 'Mexican restaurant', 'Restaurant', 'Taco restaurant', 'Tex-Mex restaurant', 'Vegetarian restaurant']
    *   ['Restaurant']
    *   ['American restaurant']
    *   ['Grocery store', 'Propane supplier']

These findings indicate that large retail centers, popular restaurants (especially fast food and casual dining), and general merchandise stores tend to attract reviews from a wider customer base.

**Seasonal Patterns of Reviews:**

*   **Reviews by Year:** The number of reviews has significantly increased over the years, with a substantial jump starting around 2016 and peaking in 2019 and 2020. This suggests a growing trend of users leaving reviews or an expansion of the dataset's coverage in recent years.

* **Reviews by Month:** The months June to September have the highest review counts > 45k. This shows that most customers tend to spend more time during summer months shopping at large retail centers, restuarants and general merchandise shops. It might also be reflective customer behavioral patterns during different times of the year.

* **Reviews by day of week** The days (saturday and sunday) have more review counts > 75k, compared to the rest of the days an indicator that customers tend to spend more times shopping during the weekend that over the rest of the days.


**Answer 1.6**

**Answer 1.6.1**

**Mark down comments Pending**

**Answer 1.6.2**

In [ ]:
# Create a user-item matrix using the relevant columns from the joined DataFrame
# Select and cast the necessary columns
ratings_df = joined_df.select(
    col("user_id").cast(IntegerType()).alias("userId"),
    col("gmap_id").alias("itemId"),
    col("rating").cast(DoubleType()).alias("rating")
)

# Drop rows with null values in the selected columns
ratings_df = ratings_df.dropna(subset=["userId", "itemId", "rating"])

# Convert item IDs to numerical IDs for ALS
# Hash the string IDs
ratings_df = ratings_df.withColumn("itemIdNumeric", hash(col("itemId")))

# Split the data into training and testing sets
(training, test) = ratings_df.randomSplit([0.8, 0.2], seed=123)

# Build the recommendation model using ALS (Alternating Least Squares)
# ALS is a common collaborative filtering algorithm suitable for sparse data
als = ALS(maxIter=5, regParam=0.09, rank=20, userCol="userId", itemCol="itemIdNumeric", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

# Generate top 10 business recommendations for each user
userRecs = model.recommendForAllUsers(10)

# Show the recommendations for a few users
print("Top 10 recommendations for users:")
userRecs.show(5, truncate=False)

# Generate top 10 user recommendations for each business
itemRecs = model.recommendForAllItems(10)

# Show the recommendations for a few businesses
print("Top 10 user recommendations for businesses:")
itemRecs.show(5, truncate=False)



**Answer 1.7**

**Answer 1.7.1**

In [ ]:
# Group by category and calculate the average rating
avg_rating_by_category = joined_df.groupBy("category").agg(avg("rating").alias("average_rating"))

# Order by average_rating in descending order
avg_rating_by_category = avg_rating_by_category.orderBy(col("average_rating").desc())

# Convert to pandas DataFrame for plotting
avg_rating_by_category_pd = avg_rating_by_category.toPandas()

# Select the top 20 categories by average rating for better visualization
top_20_avg_rating_categories = avg_rating_by_category_pd.head(20)

# Create a bar plot
plt.figure(figsize=(20, 15))
sns.barplot(data=top_20_avg_rating_categories, x='category', y='average_rating', palette='viridis')
plt.title('Top 20 Business Categories by Average Rating')
plt.xlabel('Business Category')
plt.ylabel('Average Rating')
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Explore the categories with the lowest average ratings
bottom_20_avg_rating_categories = avg_rating_by_category_pd.tail(20)

plt.figure(figsize=(20, 10))
sns.barplot(data=bottom_20_avg_rating_categories, x='category', y='average_rating', palette='viridis')
plt.title('Bottom 20 Business Categories by Average Rating')
plt.xlabel('Business Category')
plt.ylabel('Average Rating')
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()

### Insights from Relationship between Rating and Business Categories

The visualizations above illustrate the relationship between business categories and their average ratings.

**Top 20 Business Categories by Average Rating:**

The first bar plot displays the top 20 business categories with the highest average ratings. Observing this plot, we can identify categories that consistently receive positive reviews from users. These might include niche services, specialized shops, or attractions that cater to specific interests and potentially exceed customer expectations within those areas. It's worth noting that some categories might have a smaller number of reviews contributing to their high average, which could make the average less representative.

**Bottom 20 Business Categories by Average Rating:**

The second bar plot shows the bottom 20 business categories with the lowest average ratings. This plot helps pinpoint categories that tend to receive lower ratings. These could be types of businesses that are more prone to customer complaints, have inherent challenges in meeting expectations, or face strong competition leading to critical reviews. Understanding these categories can be valuable for businesses within them to identify areas for improvement.

**General Observations:**

*   The plots reveal a variation in average ratings across different business categories.
*   Some categories seem to consistently perform better in terms of customer satisfaction (based on average rating) than others.
*   To gain deeper insights, it would be beneficial to also consider the number of reviews for each category when interpreting the average ratings, as categories with very few reviews might have skewed averages.
*   Further analysis could involve examining the distribution of ratings within specific categories or exploring the common themes in the reviews for high- and low-rated categories.

**Answer 1.7.2**

In [ ]:
# Define a threshold for low ratings (e.g., ratings less than or equal to 2)
low_rating_threshold = 2

# Filter the joined DataFrame to get reviews with low ratings
low_rated_reviews = joined_df.filter(col("rating") <= low_rating_threshold)

# Count the number of low-rated reviews
num_low_rated_reviews = low_rated_reviews.count()

print(f"Number of reviews with a rating of {low_rating_threshold} or less: {num_low_rated_reviews}")

# Extract the text from the low-rated reviews and combine it into a single string
low_rated_text_df = low_rated_reviews.select(concat_ws(" ", col("text"))).collect()
low_rated_text = low_rated_text_df[0][0] if low_rated_text_df else ""

# Process the text: remove punctuation, convert to lowercase, and split into words
if low_rated_text:
    low_rated_words = low_rated_reviews.select(lower(regexp_replace(col("text"), r'[^\w\s]', '')).alias("text"))
    low_rated_words = low_rated_words.select(explode(split(col("text"), "\s+")).alias("word"))

    # Remove common English stop words and words with length less than or equal to 1
    stop_words_extended = stop_words.union({"no", "review"}) # Add "no" and "review" to stop words
    low_rated_words = low_rated_words.filter(~col("word").isin(stop_words_extended)).filter(length(col("word")) > 1)

    # Count the frequency of each word
    low_rated_word_counts_df = low_rated_words.groupBy("word").count().orderBy(col("count").desc())

    # Get the top 30 most common words in low-rated reviews
    top_30_low_rated_words = low_rated_word_counts_df.limit(30).collect()

    print("\nTop 30 most common words in low-rated reviews:")
    for row in top_30_low_rated_words:
        print(f"{row['word']}: {row['count']}")
else:
    print("\nNo low-rated reviews found to analyze text.")

### **Analysis of Low-Rated Reviews**

Based on the analysis of reviews with a rating of 2 or less:

- There are **36884** reviews with a low rating.
- The most common words appearing in these low-rated reviews include:


**food**: This suggests a product indicating poor food quality or service

**service**: This may relate to the quality of service, mostly poor services offered.

**get**: This could relate to difficulty in receiving service or products

**time**: This suggests that issues related to waiting times or the duration of service might be a common complaint.

**like**: Can be used in various contexts, but in negative reviews, it might express disappointment or unfavorable comparisons.

**bad**: A direct expression of dissatisfaction.

**never**: Used to emphasize a consistently negative experience.

**store**: Many low ratings are for retail establishments.

**went**: Describes negative experiences encountered during a visit.

**dont**: Indicates negative experiences or lack of something expected.

These words collectively suggest that common reasons for low ratings include issues with service speed and efficiency, poor customer service and communication, problems with products or work quality, and negative experiences related to specific types of businesses like restaurants and retail stores.

**Answer 1.8**

**Answer 1.8.1**

In [ ]:
# Sort reviews for each business by newtime
window_spec_business = Window.partitionBy("gmap_id").orderBy(asc("newtime"))
sorted_reviews_by_business = joined_df.withColumn("row_number", rank().over(window_spec_business)).drop("row_number")

# Group by user_id and collect the list of business names
user_business_list_df = sorted_reviews_by_business.groupBy("user_id").agg(collect_list("business_name").alias("business_list"))

# Convert the result to a list of (user_id, business_list) tuples (optional, for display)
user_business_list = user_business_list_df.collect()

# Display the result for a few users
print("Business list for each user (first 20 users):")
for row in user_business_list[:20]:
    print(f"User ID: {row['user_id']}, Businesses: {row['business_list']}")

**Answer 1.8.2**

In [ ]:
# Function to check for repeated business names in a list
def has_repeated_businesses(business_list):
    if not business_list:
        return False
    # Convert the list to a set to find unique businesses
    unique_businesses = set(business_list)
    # If the number of unique businesses is less than the total number of reviews,
    # it means there are repeated business names
    return len(unique_businesses) < len(business_list)

# Count the number of users with repeated business names in their list
users_with_repeated_businesses_count = sum(
    1 for user_id, business_list in user_business_list if has_repeated_businesses(business_list)
)

print(f"Number of users with repeated business names in their review history: {users_with_repeated_businesses_count}")

In [ ]:
# Identify and print duplicated business names and their counts for each reviewer
print("\nDuplicated business names and their counts for each user:")
count = 0
for user_id, business_list in user_business_list:
    if has_repeated_businesses(business_list):
        # Use Counter to find duplicated business names and their counts
        business_counts = Counter(business_list)
        duplicated_businesses = {business: count for business, count in business_counts.items() if count > 1}
        if duplicated_businesses:
            print(f"User ID: {user_id}")
            count += 1
            for business, business_count in duplicated_businesses.items():
                if count <= 1000:  # Limit to the first 1000 lines
                    print(f"  - {business}: {business_count} times")
                    count += 1
                else:
                    break # Exit the inner loop if 1000 lines are reached
    if count > 1000: # Exit the outer loop if 1000 lines are reached
        break

In [ ]:
# Create a new list to store user_id and a list of unique businesses
user_business_list_unique = []

# Iterate through the original list
for user_id, business_list in user_business_list:
    # Convert the business list to a set to get unique businesses, then convert back to a list
    unique_businesses = list(set(business_list))
    # Append the user_id and the list of unique businesses to the new list
    user_business_list_unique.append((user_id, unique_businesses))

# Display the result for a few users to verify
print("Business list for each user with duplicates removed (first 20 users):")
for row in user_business_list_unique[:20]:
    print(f"User ID: {row[0]}, Businesses: {row[1]}")

**Answer 1.8.3**

In [ ]:
from pyspark.sql.functions import udf,expr
from pyspark.sql.types import IntegerType

# Define a UDF to encode business names using hashing
@udf(IntegerType())
def encode_business_name(business_name):
    if business_name is None:
        return None
    return hash(business_name)

In [ ]:
# Apply the hash function directly to each element in the business_list column
encoded_user_business_list_df = user_business_list_df.withColumn(
    "encoded_business_list",
    # Use transform to apply the hash function to each element in the list
    expr("transform(business_list, business -> hash(business))")
)

# Display the result for a few users to verify
print("Business list for each user with encoded business names (first 20 users):")
for row in encoded_user_business_list_df.collect()[:20]:
    print(f"User ID: {row['user_id']}, Encoded Businesses: {row['encoded_business_list']}")

In [ ]:
encoded_user_business_list_df.columns

In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations
from collections import defaultdict

# NOTE: This assumes 'df' is the pandas DataFrame created in the notebook (Cell 15)

# Group the DataFrame by user_id and collect the unique gmap_id's into a set for fast lookups
encoded_user_business_list_series = df.groupby('user_id')['gmap_id'].apply(lambda x: set(x.unique()))

# Convert the Series back to a DataFrame with a descriptive column name
encoded_user_business_list_df = encoded_user_business_list_series.rename('reviewed_businesses').to_frame()

print("--- Encoded User-Business List (Sample) ---")
display(encoded_user_business_list_df.head())

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict

# --- Jaccard Similarity Function (Unchanged, it is efficient) ---
def calculate_jaccard_similarity(set_a, set_b):
    """
    Calculates the Jaccard similarity between two sets.
    """
    # Calculate intersection and union size
    intersection = len(set_a.intersection(set_b))
    union = len(set_a.union(set_b))

    if union == 0:
        return 0.0
    return intersection / union

# --- Calculate Pairwise Similarities (Refactored for Sparsity) ---

# Get the list of all unique users and their business sets
# NOTE: user_sets must be a Series/Dictionary mapping user_id to a set of business IDs.
# For example: user_sets = encoded_user_business_list_series
user_sets = encoded_user_business_list_series # Assuming this is available
user_ids = user_sets.index.tolist()

# ⚠️ MEMORY SAVING CHANGE: Use a list to store only non-zero similarities
# This avoids allocating space for all N*N zero-similarity pairs.
sparse_similarity_data = []

# Iterate through all unique pairs of users
for user_a, user_b in combinations(user_ids, 2):
    set_a = user_sets.loc[user_a]
    set_b = user_sets.loc[user_b]

    similarity = calculate_jaccard_similarity(set_a, set_b)

    # ⚠️ CRITICAL STEP: Store only pairs with positive similarity (they share at least one business)
    if similarity > 0:
        # Store symmetrically: (A, B, sim) and (B, A, sim)
        sparse_similarity_data.append({'user_a': user_a, 'user_b': user_b, 'similarity': similarity})
        sparse_similarity_data.append({'user_a': user_b, 'user_b': user_a, 'similarity': similarity})

# Convert the list of sparse results into a Pandas DataFrame
sparse_similarity_df = pd.DataFrame(sparse_similarity_data)

# Add self-similarity (1.0) for every user
# This is required if the system needs to recommend items from the user's own history
self_similarity_data = [{'user_a': uid, 'user_b': uid, 'similarity': 1.0} for uid in user_ids]
sparse_similarity_df = pd.concat([sparse_similarity_df, pd.DataFrame(self_similarity_data)], ignore_index=True)

print("\n--- Sparse User Similarity Data (Sample) ---")
display(sparse_similarity_df.head())

Jaccard similarity is a statistic used for comparing the similarity and diversity of sample sets. In this context, we're using it to compare the sets of businesses reviewed by different customers.

The strategy used is as follows:

Representing Businesses as Sets: For each user, we create a set of the unique businesses they have reviewed using the encoded business names.

Calculating Intersection: The intersection of two users' sets includes the businesses that both users have reviewed.

Calculating Union: The union of two users' sets includes all unique businesses that either user has reviewed.

Jaccard Similarity Formula: The Jaccard similarity is calculated as the size of the intersection divided by the size of the union:

Jaccard Similarity (User A, User B) = |Set of businesses reviewed by A ∩ Set of businesses reviewed by B| / |Set of businesses reviewed by A ∪ Set of businesses reviewed by B|

A Jaccard similarity of 1 means the two users have reviewed the exact same set of businesses, while a similarity of 0 means they have reviewed completely different sets of businesses. Values in between indicate varying degrees of overlap in their review history.

In the code, we are calculating this similarity based on the TF-IDF vectors derived from the encoded business names. The calculate_jaccard_similarity function does this by looking at the indices of the non-zero elements in the sparse vectors, which correspond to the businesses present in the user's review history.

**Answer 2**

In [ ]:
# Cell 0 — Packages: install and import (simple and annotated)
# Why: set up a small, standard toolbox for time series work:
# - numpy/pandas/matplotlib/seaborn for data and plots
# - statsmodels for classical TS models and diagnostics
# - scikit-learn for metrics and simple baselines
# - prophet for later structural forecasting
# Notes: keep imports lean and human-readable.

# Make sure any old "fbprophet" is removed and Prophet is available
%pip -q install prophet

# Core scientific stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Classical time series (statsmodels)
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima.model import ARIMA

# ML utilities
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

# Prophet (structural forecasting)
from prophet import Prophet

# Spark helpers (for aggregation in the next cell)
from pyspark.sql import functions as F

# Plot style
%matplotlib inline
sns.set(style="whitegrid")

# References:
# Hunter, J.D. (2007) Matplotlib: A 2D graphics environment. Computing in Science & Engineering, 9(3), 90–95.
# McKinney, W. (2010) Data structures for statistical computing in Python. In: Proceedings of the 9th Python in Science Conference, pp. 51–56.
# Waskom, M. (2021) seaborn: statistical data visualization. Journal of Open Source Software, 6(60), 3021.
# Seabold, S. and Perktold, J. (2010) statsmodels: Econometric and statistical modeling with Python. Proceedings of the 9th Python in Science Conference.
# Pedregosa, F. et al. (2011) Scikit-learn: Machine learning in Python. Journal of Machine Learning Research, 12, 2825–2830.
# Taylor, S.J. and Letham, B. (2018) Forecasting at scale. The American Statistician, 72(1), 37–45.

In [ ]:
# Cell 1 — Select dataframe and prepare it for analysis
# Task: from joined_df, compute daily review counts and make a clean pandas time series.
# Why: most TS methods expect a regular date index with one value per day.
# Method: group by date (newtime), count records, convert to pandas, set DateTimeIndex.
# Cite: aggregation of counts to a regular TS is standard in classical workflows (Box et al., 2015).

# Safety: check joined_df exists
try:
    _ = joined_df.printSchema()
except NameError:
    raise RuntimeError("joined_df not found. Please run the earlier join cell to create it.")

# 1) Aggregate to reviews per day in Spark
daily_counts_spark = (
    joined_df
    .where(F.col("newtime").isNotNull())
    .groupBy("newtime")
    .agg(F.count(F.lit(1)).alias("reviews"))
    .orderBy("newtime")
)

# 2) Move to pandas and set a clean daily index
daily_pdf = daily_counts_spark.toPandas()
daily_pdf["newtime"] = pd.to_datetime(daily_pdf["newtime"])
daily_pdf = daily_pdf.sort_values("newtime").set_index("newtime")

print("First 5 observed days (before any filling):")
display(daily_pdf.head())

print("\nQuick shape and date span:")
print("rows:", len(daily_pdf), "| cols:", list(daily_pdf.columns))
print("range:", daily_pdf.index.min().date(), "to", daily_pdf.index.max().date())

# References (Harvard style):
# Box, G.E.P., Jenkins, G.M., Reinsel, G.C. and Ljung, G.M. (2015) Time Series Analysis: Forecasting and Control. 5th edn. Hoboken, NJ: Wiley.

**Answer 2.1**

In [ ]:

# Standardize the date column (convert to datetime and normalize to date only)
df['review_date'] = pd.to_datetime(df['review_date']).dt.normalize()

# Create the daily time series by counting reviews per day
daily_reviews = df.groupby('review_date').size().rename('review_count')

# Determine the full date range of the dataset
min_date = daily_reviews.index.min()
max_date = daily_reviews.index.max()
full_date_range = pd.date_range(start=min_date, end=max_date, freq='D')

# Reindex the time series to include all days in the range (this introduces NaNs for missing days)
full_ts = daily_reviews.reindex(full_date_range)

# Calculate the mean review count for imputation: Total Reviews / Total Days in Range
total_reviews = daily_reviews.sum()
total_days = len(full_date_range)
mean_reviews_imputation = total_reviews / total_days

# Fill the missing days (NaNs) with the calculated mean
imputed_ts = full_ts.fillna(mean_reviews_imputation)
imputed_ts.index.freq = 'D' # Set frequency for decomposition

print("--- Imputation Details ---")
print(f"Total Reviews: {total_reviews}")
print(f"Total Days in Full Range: {total_days}")
print(f"Calculated Mean Reviews per Day (Imputation Value): {mean_reviews_imputation:.2f}")
print(f"Number of Missing Days Filled: {len(daily_reviews) - len(imputed_ts.dropna())}")

# Decompose the time series using the Additive Model with a weekly period (7 days)
decomposition = seasonal_decompose(imputed_ts, model='additive', period=7)

#Analysis and Plotting ---
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
decomposition.observed.plot(ax=axes[0], title='Observed Review Submissions (Daily)')
decomposition.trend.plot(ax=axes[1], title='Trend Component')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal Component (Weekly)')
decomposition.resid.plot(ax=axes[3], title='Residual Component')
plt.tight_layout()
plt.show()

**Answer 2.2**

In [ ]:

# Split the Time Series (80% Train, 20% Test)
train_size = int(len(imputed_ts) * 0.8)
train, test = imputed_ts[:train_size], imputed_ts[train_size:]
print(f"Training set size: {len(train)}")
print(f"Testing set size: {len(test)}")

#Define the Grid Search Parameters
p_params = d_params = q_params = [0, 1, 2]
pdq_combinations = list(product(p_params, d_params, q_params))

best_mae = float('inf')
best_order = None
results = []

print("\n--- Starting ARIMA Grid Search (p, d, q in [0, 1, 2]) ---")

#Grid Search Loop
for order in pdq_combinations:
    try:
        # Train the ARIMA Model
        model = ARIMA(train, order=order)
        model_fit = model.fit()

        # Forecast
        # Start and end indices for the forecast period (should match the test set)
        start_index = len(train)
        end_index = len(imputed_ts) - 1
        forecast = model_fit.predict(start=start_index, end=end_index)

        # Calculate MAE
        mae = mean_absolute_error(test, forecast)
        results.append({'order': order, 'mae': mae})

        # Update Best Model
        if mae < best_mae:
            best_mae = mae
            best_order = order
            # Print the new best model as it's found
            print(f"-> New Best: ARIMA{order} - MAE: {mae:.2f}")

    except Exception as e:
        # Skip models that fail to converge or raise other errors
        continue

# Final Output
results_df = pd.DataFrame(results).sort_values(by='mae')

print("\n--- Grid Search Complete ---")
print(f"Total models tested: {len(results)}")
print(f"Best ARIMA Model: ARIMA{best_order} with MAE: {best_mae:.4f}")
print("\nTop 5 Models by MAE:")
display(results_df.head())